# Start to Finish

# Imports

In [ ]:
%matplotlib inline
import librosa, librosa.display
import numpy as np
import random 
import os
import matplotlib.pyplot as plt
import math
#from spleeter.separator import Separator

import moviepy.editor as mp
from PIL import Image
from datetime import datetime

start = datetime.now()

In [ ]:
np.set_printoptions(linewidth=10000, suppress=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Params

In [ ]:
data_path = '/content/drive/MyDrive/shortcuts/Asha FM'
data_path

In [ ]:

DRUM_AUDIO_PATH = '/content/drive/MyDrive/Asha FM/raw_data/audio/MDU.mp3'
audio_path = '/content/drive/MyDrive/Asha FM/raw_data/audio/MDU.mp3'
AUDIO_FILE_NAME = 'AUDIO_FILE'
RAW_AUDIO_PATH = '/content/drive/MyDrive/Asha FM/raw_data/audio/MDU.mp3'


In [ ]:
# IO
# AUDIO_FILE_NAME = RAW_AUDIO_PATH.split('/')[-1].split('.')[0]

# SPLIT_AUDIO_PATH = f"../raw_data/audio/{AUDIO_FILE_NAME}/"
# DRUM_AUDIO_PATH = f"{SPLIT_AUDIO_PATH}{AUDIO_FILE_NAME}-drums.wav"

VIDEO_SAVE_NAME = f"{AUDIO_FILE_NAME}-asha.mp4"
VIDEO_SAVE_PATH = f"../outputs/{VIDEO_SAVE_NAME}"

# GLOBAL
# FIXED DURATION

# AUDIO 
FRAME_RATE = 60
HOP_WINDOW = 512

# Images
IMAGE_TIME_GAP = 1 # seconds, play with this
IMAGE_FRAME_GAP = IMAGE_TIME_GAP * FRAME_RATE

# VIDEO
MAX_VIDEO_DURATION = 19

# Audio

## Split the Audio into stems

With spleeter by deezer

## Extracting Audio info from drum stem

### Extraction functions

In [ ]:
def get_audio_params(audio_path):
    # Load song with 22050 sample rate
    y, sr = librosa.load(audio_path, sr=22050)
    audio_duration = librosa.get_duration(y=y, sr=sr)
    
    total_frames_float = audio_duration * FRAME_RATE
    total_frames = math.ceil(total_frames_float)
    
    sample_rate = round(total_frames_float / audio_duration * HOP_WINDOW)
    
    return total_frames, sample_rate, audio_duration

In [ ]:
def get_onset_info(audio_path, sample_rate):
    # Load
    y, sr = librosa.load(audio_path, sr=sample_rate)
    
    # Onset strengths and normalize
    onset_strengths = librosa.onset.onset_strength(y=y, sr=sample_rate, aggregate=np.median)
    onset_strengths = librosa.util.normalize(onset_strengths)
    
    # Onset timestamps and frames
    onset_times = librosa.times_like(onset_strengths, sr=sample_rate)
    onset_frames = onset_times * FRAME_RATE
    
    onset_info = np.concatenate([
        onset_frames.reshape((-1, 1)),
        onset_times.reshape((-1, 1)),
        onset_strengths.reshape((-1, 1))
    ], axis=1)
    
    # Beat times
    beat_times = librosa.beat.beat_track(y=y, sr=sample_rate, units='time')[1]
    
    return y, onset_info, beat_times

### Using those functions

In [ ]:
total_frames, sample_rate, audio_duration = get_audio_params(DRUM_AUDIO_PATH)
total_frames, sample_rate, audio_duration

In [ ]:
y, onset_info, beat_times = get_onset_info(DRUM_AUDIO_PATH, sample_rate)
y.shape, onset_info.shape, beat_times.shape

## Create additional onset strength features

- Linear Decay
- Exponential Decay

In [ ]:
def create_onset_features(onset_info, beat_times, lin_decay_time=0.25, exp_decay_rate=0.25, decay_magnification=True):
    
    fixed_decay_frames = int(lin_decay_time * FRAME_RATE)
    
    # Create column of zeroes as default value for both Linear and Exp Decay
    onset_info = np.concatenate([onset_info, np.zeros((onset_info.shape[0], 1)), np.zeros((onset_info.shape[0], 1))], axis=1)
    
    # for each row
    for i in range(onset_info.shape[0]):
        # Skip the first value, makes life easy
        if i == 0:
            onset_info[i, 3] = 0
        
        
        ## LINEAR
        # If the timestamp is in beat_times, it's a peak
        if onset_info[i, 1] in beat_times:
            onset_info[i, 3] = onset_info[i, 2] # Linear Column
            
            # Decay Params
            decay_factor = 1
            if decay_magnification:
                decay_factor *= (onset_info[i, 2] + 1)
            
            decay_frames = fixed_decay_frames * decay_factor
            lin_decay_val = onset_info[i, 2] / decay_frames
        
        # Check if the previous value is zero or less than the decay_val -> 0
        if onset_info[i - 1, 3] == 0. or abs(onset_info[i -1, 3]) < lin_decay_val:
            pass
        
        # If previous value > 0, needs decay
        elif onset_info[i - 1, 3] > 0: 
            onset_info[i, 3] = onset_info[i - 1, 3] - lin_decay_val
        
        
        # EXPONENTIAL
        # If the timestamp is in beat_times, it's a peak
        if onset_info[i, 1] in beat_times:
            onset_info[i, 4] = onset_info[i, 2] # Exp Column
        
        # Set current to zero if previous is zero or small number
        elif onset_info[i - 1, 4] == 0 or onset_info[i - 1, 4] < 0.005: 
            pass
        
        # If previous value > 0, needs decay
        elif onset_info[i - 1, 4] != 0: 
                onset_info[i, 4] = onset_info[i - 1, 4] * (1 - exp_decay_rate)
    
    return onset_info

In [ ]:
y, onset_info, beat_times = get_onset_info(DRUM_AUDIO_PATH, sample_rate)
onset_info = create_onset_features(onset_info, beat_times, lin_decay_time=0.25, exp_decay_rate=0.25, decay_magnification=False)

# Model Stuff

In [ ]:
import numpy as np
from PIL import Image
import os

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Reshape, Conv2D, Conv2DTranspose, Dense, Flatten, MaxPooling2D
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
#ds = image_dataset_from_directory('../raw_data/zelle', label_mode=None, image_size=(448, 448), batch_size=16)

In [ ]:
#ds = ds.map(lambda x: x/255)

In [ ]:
# imgs = ds.as_numpy_iterator().__next__()
# imgs.shape

### Model Architecture

Because we are using load weights, we need to instantiate the model every time. This sucks for notebooks, but it will be fine once everything is packaged.

In [ ]:
class Sampling(tf.keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
latent_dim = 200

## Encoder

input_image = Input(shape=(448, 448, 3))
    
x = Conv2D(32, (3, 3), padding='same', activation="relu")(input_image)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), padding='same', activation="relu")(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), padding='same', activation="relu")(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(256, (3, 3), padding='same', activation="relu")(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)

z_mean = Dense(latent_dim, name="z_mean")(x)
z_log_var = Dense(latent_dim, name="z_log_var")(x)

z = Sampling()([z_mean, z_log_var])

# Encoder Build
encoder = Model(input_image, [z_mean, z_log_var, z], name="encoder")

#############################################

## Decoder

latent_inputs = Input(shape=(latent_dim,))  
y = Dense(7*7*128, activation='tanh')(latent_inputs)
y = Reshape((7, 7, 128))(y)

y = Conv2DTranspose(256, (3, 3), strides=2, padding='same', activation="relu")(y)
y = Conv2DTranspose(256, (3, 3), strides=1, padding='same', activation="relu")(y)
y = Conv2DTranspose(256, (3, 3), strides=1, padding='same', activation="relu")(y)
y = Conv2DTranspose(256, (3, 3), strides=1, padding='same', activation="relu")(y)

y = Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation="relu")(y)
y = Conv2DTranspose(128, (3, 3), strides=1, padding='same', activation="relu")(y)
y = Conv2DTranspose(128, (3, 3), strides=1, padding='same', activation="relu")(y)
y = Conv2DTranspose(128, (3, 3), strides=1, padding='same', activation="relu")(y)

y = Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation="relu")(y)
y = Conv2DTranspose(64, (3, 3), strides=1, padding='same', activation="relu")(y)
y = Conv2DTranspose(64, (3, 3), strides=1, padding='same', activation="relu")(y)

y = Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation="relu")(y)
y = Conv2DTranspose(32, (3, 3), strides=1, padding='same', activation="relu")(y)
y = Conv2DTranspose(32, (3, 3), strides=1, padding='same', activation="relu")(y)

y = Conv2DTranspose(16, (3, 3), strides=2, padding='same', activation="relu")(y)
y = Conv2DTranspose(16, (3, 3), strides=1, padding='same', activation="relu")(y)
y = Conv2DTranspose(16, (3, 3), strides=1, padding='same', activation="relu")(y)

decoder_output = Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='sigmoid')(y)

# Decoder Build
decoder = Model(inputs=latent_inputs, outputs=decoder_output, name="decoder")

In [ ]:
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

## Instantiate Model and Load weights

In [ ]:
vae = VAE(encoder, decoder)

vae.load_weights('/content/drive/MyDrive/Asha FM/raw_data/model/e5100/vae_complex_model_epoch5100')

### Predict

Loads encoded_images numpy file

In [ ]:
filename = "/content/drive/MyDrive/Asha FM/raw_data/images/encoded_images.npy"

def load_images_from_file(filename):
    latent_space = np.load(filename)
    return latent_space

In [ ]:
N_IMAGES = math.ceil(audio_duration / IMAGE_TIME_GAP + 1)
N_IMAGES

In [ ]:
encoded_images = load_images_from_file("/content/drive/MyDrive/Asha FM/raw_data/images/encoded_images.npy")

In [ ]:
#encoded_images = vae.decoder.predict(encoded_images[0:3 + N_IMAGES])[2]

In [ ]:
encoded_images.shape

In [ ]:
random_start_point = np.random.randint(0, encoded_images.shape[0] - N_IMAGES, size=(1))[0]
random_start_point

# Create Interpolated Images

In [ ]:
def create_interp_vecs(N_IMAGES, encoded_images, min_factor=0.01, onset_info=onset_info):
    # Empty lists to append new vectors to
    lin_interp_vecs = []
    exp_interp_vecs = []
    
    # We want to interate through the images to be able to interpolate between them
    for i in range(N_IMAGES - 1):
        
        # Get start and stop latent vectors
        start_vec = encoded_images[i]
        end_vec = encoded_images[i + 1]
        
        # get step_vec between image_n and image_n + 1
        step_vec = (end_vec - start_vec) / IMAGE_FRAME_GAP
            
        # We now need to create an image at every frame 
        # Mad props to Charlotte for making this way simpler than I was trying to make it
        for j in range(IMAGE_FRAME_GAP):
            
            current_frame = IMAGE_FRAME_GAP * i + j
            # Check to see if current_frame > total_frames : no point making extra images
            if current_frame > int(onset_info[-1,0]):
                break
            lin_interp_vecs.append(start_vec + (step_vec * j * (onset_info[current_frame, 3] / min_factor + 1)))
            exp_interp_vecs.append(start_vec + (step_vec * j * (onset_info[current_frame, 4] / min_factor + 1)))
            
    return np.array(lin_interp_vecs), np.array(exp_interp_vecs)

In [ ]:
lin_interp_vecs, exp_interp_vecs = create_interp_vecs(N_IMAGES, encoded_images, min_factor=1, onset_info=onset_info)

In [ ]:
# Hyperparameters for create onset info
lin_decay_time_list = [0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7]
exp_decay_rate_list = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]

onset_info_dict = {}

for i, lin_decay_time in enumerate(lin_decay_time_list):
    for j, exp_decay_rate in enumerate(exp_decay_rate_list):
        onset_info_dict[(lin_decay_time, exp_decay_rate)] = create_onset_features(onset_info, beat_times, lin_decay_time, exp_decay_rate)
for key, value in onset_info_dict.items():
    print(f"{key}: {value}")

In [ ]:
# Define a list of min_factor values to use
min_factor_list = [0.01, 0.05, 0.1, 0.25, 0.35, 0.5, 0.6, 0.75, 0.9, 0.99]

# Define empty dictionaries to store the results
lin_interp_vecs_dict = {}
exp_interp_vecs_dict = {}

# Loop over the list of min_factors and call create_interp_vecs function for each
for min_factor in min_factor_list:
    lin_interp_vecs, exp_interp_vecs = create_interp_vecs(N_IMAGES, encoded_images, min_factor, onset_info)
    lin_interp_vecs_dict[f"lin_interp_vecs_{min_factor}"] = lin_interp_vecs
    exp_interp_vecs_dict[f"exp_interp_vecs_{min_factor}"] = exp_interp_vecs

# Print the dictionaries
print(lin_interp_vecs_dict)
print(exp_interp_vecs_dict)

In [ ]:
lin_interp_vecs.shape, exp_interp_vecs.shape

In [ ]:
import keras.backend as K
import os

save_path = '/content/drive/MyDrive/Asha FM/raw_data/images/processed_images'


def create_interp_imgs(decoder, lin_interp_vecs, exp_interp_vecs, batch_size=1, save_path=save_path):
    # Linear Images
    batch_memory = 500 * 1024 * 1024  # 500 MB

    num_lin_imgs = len(lin_interp_vecs)
    lin_interp_imgs = []
    for i in range(0, num_lin_imgs, batch_size):
        batch_vecs = lin_interp_vecs[i:i+batch_size]
        batch_imgs = decoder.predict(batch_vecs)
        batch_imgs = (batch_imgs * 255).astype('uint8')
        lin_interp_imgs.append(batch_imgs)
        print(f"Processed {i+batch_size} linear images")
        if save_path and i % (500 * 1024 * 1024 // batch_memory) == 0:
            if len(lin_interp_imgs) > 0:
                np.save(os.path.join(save_path, f"lin_interp_imgs_{i}.npy"), np.concatenate(lin_interp_imgs, axis=0))
                lin_interp_imgs = []
        K.clear_session()
    if len(lin_interp_imgs) > 0:
        lin_interp_imgs = np.concatenate(lin_interp_imgs, axis=0)
        if save_path:
            np.save(os.path.join(save_path, f"lin_interp_imgs_{num_lin_imgs}.npy"), lin_interp_imgs)
    
    # Exponential Images
    num_exp_imgs = len(exp_interp_vecs)
    exp_interp_imgs = []
    for i in range(0, num_exp_imgs, batch_size):
        batch_vecs = exp_interp_vecs[i:i+batch_size]
        batch_imgs = decoder.predict(batch_vecs)
        batch_imgs = (batch_imgs * 255).astype('uint8')
        exp_interp_imgs.append(batch_imgs)
        print(f"Processed {i+batch_size} exponential images")
        if save_path and i % (500 * 1024 * 1024 // batch_memory) == 0:
            if len(exp_interp_imgs) > 0:
                np.save(os.path.join(save_path, f"exp_interp_imgs_{i}.npy"), np.concatenate(exp_interp_imgs, axis=0))
                exp_interp_imgs = []
        K.clear_session()
    if len(exp_interp_imgs) > 0:
        exp_interp_imgs = np.concatenate(exp_interp_imgs, axis=0)
        if save_path:
            np.save(os.path.join(save_path, f"exp_interp_imgs_{num_exp_imgs}.npy"), exp_interp_imgs)
    
    return lin_interp_imgs, exp_interp_imgs


In [ ]:
lin_imgs, exp_imgs = create_interp_imgs(vae.decoder, lin_interp_vecs=lin_interp_vecs, exp_interp_vecs=exp_interp_vecs)

In [ ]:
def concatenate_lin_interp_imgs(save_path, num_lin_imgs):
    lin_interp_imgs = []
    for i in range(num_lin_imgs):
        file_path = os.path.join(save_path, f"lin_interp_imgs_{i}.npy")
        if os.path.exists(file_path):
            lin_interp_imgs.append(np.load(file_path))
    lin_interp_imgs = np.concatenate(lin_interp_imgs, axis=0)
    return lin_interp_imgs

def concatenate_export_interp_imgs(save_path, num_exp_imgs):
    exp_interp_imgs = []
    for i in range(num_exp_imgs):
        file_path = os.path.join(save_path, f"exp_interp_imgs_{i}.npy")
        if os.path.exists(file_path):
            exp_interp_imgs.append(np.load(file_path))
    exp_interp_imgs = np.concatenate(exp_interp_imgs, axis=0)
    return exp_interp_imgs

In [ ]:
num_lin_imgs = len(lin_interp_vecs)
num_exp_imgs = len(exp_interp_vecs)

interp_imgs = concatenate_lin_interp_imgs(save_path, num_lin_imgs)
exp_imgs = concatenate_lin_interp_imgs(save_path, num_exp_imgs)

# Create AV

In [ ]:
def create_video_with_audio(interp_imgs):
    
    # Load the audio file
    audio_clip = mp.AudioFileClip(RAW_AUDIO_PATH)
    
    # Needed for some reason?
    frames = [mp.ImageClip(interp_img).set_duration(1/FRAME_RATE) for interp_img in interp_imgs]
    
    # Combine the frames into a video clip
    video_clip = mp.concatenate_videoclips(frames, method='chain')
    
    # Overlay the audio on the video clip
    final_clip = video_clip.set_audio(audio_clip)
    
    # Write the final video clip to file
    final_clip.write_videofile("donk.mp4", fps=FRAME_RATE)

In [ ]:
VIDEO_SAVE_PATH

In [ ]:
create_video_with_audio(interp_imgs)

In [ ]:
print(datetime.now() - start)